# **Problem Statement**  
## **23. Implement Tarjan's algorithm for strongly connected components.**

Given a directed graph, find all Strongly Connected Components (SCCs) using Tarjan’s Algorithm.
A Strongly Connected Component is a maximal subgraph in which every vertex is reachable from every other vertex.

### Constraints & Example Inputs/Outputs
#### Constraints
- Graph is directed
- Can be connected or disconnected
- Number of vertices: 1 ≤ V ≤ 10^5
- Number of edges: 0 ≤ E ≤ 2 × 10^5

Example Input:
```python
V = 5
Edges:
0 → 1
1 → 2
2 → 0
1 → 3
3 → 4
```
Expected Output
```python
SCCs:
[0, 1, 2]
[3]
[4]
```

### Solution Approach

Here are the 2 best possible approaches:
#### Brute Force Approach (Kosaraju’s Algorithm)
1. Perform DFS to compute finish times.
2. Reverse all edges.
3. Run DFS in decreasing finish-time order to extract SCCs.

Time Complexity: O(V + E) but two DFS passes + reverse graph.
We treat this as “brute force” because Tarjan is simpler and faster in practice.

#### Optimized Approach — Tarjan’s Algorithm
Tarjan performs one DFS with the following tracking variables:
- disc[u] → discovery time
- low[u] → earliest reachable ancestor
- stack → tracks current recursion path
- in_stack[u] → boolean for stack membership

#### Key Idea
A node is the head of an SCC when:

disc[u] == low[u]

Then pop from stack until u is reached → this forms an SCC.

- Time Complexity: O(V + E)
- Space Complexity: O(V)


### Solution Code

In [1]:
# Approach1: Brute Force Solution (Kosaraju’s Algorithm)
from collections import defaultdict

def kosaraju_scc(V, edges):
    graph = defaultdict(list)
    rev_graph = defaultdict(list)
    
    for u, v in edges:
        graph[u].append(v)
        rev_graph[v].append(u)
    
    visited = [False] * V
    stack = []
    
    def dfs1(u):
        visited[u] = True
        for v in graph[u]:
            if not visited[v]:
                dfs1(v)
        stack.append(u)
    
    # Step 1: Order vertices by finish time
    for i in range(V):
        if not visited[i]:
            dfs1(i)
    
    # Step 2: Reverse graph & find SCCs
    visited = [False] * V
    sccs = []
    
    def dfs2(u, comp):
        visited[u] = True
        comp.append(u)
        for v in rev_graph[u]:
            if not visited[v]:
                dfs2(v, comp)
    
    while stack:
        node = stack.pop()
        if not visited[node]:
            comp = []
            dfs2(node, comp)
            sccs.append(comp)
    
    return sccs


# Example Test
V = 5
edges = [(0,1), (1,2), (2,0), (1,3), (3,4)]
print("Kosaraju SCCs:", kosaraju_scc(V, edges))


Kosaraju SCCs: [[0, 2, 1], [3], [4]]


### Alternative Solution

In [2]:
# Approach2: Optimized Solution — Tarjan’s Algorithm
from collections import defaultdict

def tarjans_scc(V, edges):
    graph = defaultdict(list)
    for u, v in edges:
        graph[u].append(v)
    
    disc = [-1] * V         # Discovery times
    low = [-1] * V          # Low-link values
    in_stack = [False] * V
    stack = []
    
    time = 0
    sccs = []
    
    def dfs(u):
        nonlocal time
        disc[u] = low[u] = time
        time += 1
        
        stack.append(u)
        in_stack[u] = True
        
        for v in graph[u]:
            if disc[v] == -1:    # Not visited
                dfs(v)
                low[u] = min(low[u], low[v])
            elif in_stack[v]:    # Back edge
                low[u] = min(low[u], disc[v])
        
        # If u is root of SCC
        if low[u] == disc[u]:
            comp = []
            while True:
                w = stack.pop()
                in_stack[w] = False
                comp.append(w)
                if w == u:
                    break
            sccs.append(comp)
    
    for i in range(V):
        if disc[i] == -1:
            dfs(i)
    
    return sccs


# Example Test
V = 5
edges = [(0,1), (1,2), (2,0), (1,3), (3,4)]
print("Tarjan SCCs:", tarjans_scc(V, edges))


Tarjan SCCs: [[4], [3], [2, 1, 0]]


### Alternative Approaches

1. Using BFS-based condensation graph + repeated DFS
- Not efficient for large graphs.

2. Path-based algorithm (Gabow’s Algorithm)
- Another single-pass SCC algorithm similar to Tarjan
- Runs in O(V + E) but uses two stacks.

3. Using strongly connected components via SAT-style reachability
- Overkill → mainly useful in compilers / symbolic analysis.

### Test Case

In [ ]:
# Test Case1
V = 5
Edges:
0→1, 1→2, 2→0, 1→3, 3→4
Expected:
[0,1,2], [3], [4]

# Test Case 2
V = 4
Edges:
0→1, 1→2, 2→3
Expected:
[0], [1], [2], [3]

# Test Case 3
V = 7
Edges:
0→1, 1→2, 2→0,
1→3,
3→4, 4→5, 5→3,
5→6
Expected:
[0,1,2], [3,4,5], [6]

# Test Case 4
V = 3
Edges:
0→1, 1→2, 2→1
Expected:
[1,2], [0]

## Complexity Analysis

| Algorithm                  | Time Complexity | Space Complexity | DFS Passes |
| -------------------------- | --------------- | ---------------- | ---------- |
| **Kosaraju (Brute Force)** | `O(V + E)`      | `O(V)`           | 2 passes   |
| **Tarjan (Optimized)**     | `O(V + E)`      | `O(V)`           | 1 pass     |
| **Gabow**                  | `O(V + E)`      | `O(V)`           | 1 pass     |

#### Thank You!!